In [1]:
import numpy as np
import pandas as pd
import os
import sys
from math import frexp, ldexp
import math
import time
from condTE_utils import CondTEUtils as CTU

In [2]:
# load data
datapath = '../Firearms'
data_original = pd.read_excel(os.path.join(datapath,'summary_data.xlsx'))
BC = data_original['Background checks (48 states, processed)'].values
MS = data_original['Mass shooting'].values
MO = data_original.values[:,2:5]


In [3]:
# symbolic
XBC = (np.diff(BC,axis=0) > 0) * 1
XMS = (MS[:-1] > 0) * 1
XMO = (np.diff(MO,axis=0) > 0) * 1
Nt = len(XBC)
XBC.shape,XMS.shape,XMO.shape

((227,), (227,), (227, 3))

In [15]:
nsurr = 500

In [16]:
# mutual information (MO_laws)
mutual_info_MOlaws_label = np.array(['MO_laws->BC', 'MO_laws->MS', 'BC->MS',
                               'BC->MO_laws', 'MS->BC', 'MS->MO_laws'])
mutual_info_MOlaws_values = np.array([CTU.cal_mutual_information(XBC,XMO[:,0]),
                                      CTU.cal_mutual_information(XMS,XMO[:,0]),
                                      CTU.cal_mutual_information(XMS,XBC),
                                      CTU.cal_mutual_information(XMO[:,0],XBC),
                                      CTU.cal_mutual_information(XBC,XMS),
                                      CTU.cal_mutual_information(XMO[:,0],XMS)])
mutual_info_MOlaws = {}
mutual_info_MOlaws = {label : mutual_info_MOlaws_values[i] for i, label in enumerate(mutual_info_MOlaws_label)} 

mutual_info_MOlaws 

{'MO_laws->BC': 0.028389249277907647,
 'MO_laws->MS': 0.0007940280796410182,
 'BC->MS': 0.000875914415846557,
 'BC->MO_laws': 0.0015724634488607503,
 'MS->BC': 0.010884142586891077,
 'MS->MO_laws': 0.007692578296242036}

In [17]:
# P_values test (MO_laws)
sur_mutual_info_laws_values = [CTU.surrogate_mutual_information(XBC,XMO[:,0],nsurr),
                               CTU.surrogate_mutual_information(XMS,XMO[:,0],nsurr),
                               CTU.surrogate_mutual_information(XMS,XBC,nsurr),
                               CTU.surrogate_mutual_information(XMO[:,0],XBC,nsurr),
                               CTU.surrogate_mutual_information(XBC,XMS,nsurr),
                               CTU.surrogate_mutual_information(XMO[:,0],XMS,nsurr)]

sur_mutual_info_laws = {}
sur_mutual_info_laws = {label : sur_mutual_info_laws_values[i] for i, label in enumerate(mutual_info_MOlaws_label)}
 
p_values_laws = {}
ninetyfive_tab_laws = {}

for i, label in enumerate(mutual_info_MOlaws_label):
    aux_count_sum = np.sum(sur_mutual_info_laws[label] > mutual_info_MOlaws[label])
    p_values_laws[label] = (aux_count_sum + 1) / (nsurr + 1)
    ninetyfive_tab_laws[label] = np.quantile(sur_mutual_info_laws[label] , 0.95)
p_values_laws

{'MO_laws->BC': 0.007984031936127744,
 'MO_laws->MS': 0.6447105788423154,
 'BC->MS': 0.5888223552894212,
 'BC->MO_laws': 0.47704590818363274,
 'MS->BC': 0.0718562874251497,
 'MS->MO_laws': 0.1157684630738523}

In [18]:
# mutual information (MO_shooting)
mutual_info_MOshooting_label = np.array(['MO_shooting->BC', 'MO_shooting->MS', 'BC->MS',
                               'BC->MO_shooting', 'MS->BC', 'MS->MO_shooting'])
mutual_info_MOshooting_values = np.array([CTU.cal_mutual_information(XBC,XMO[:,2]),
                                          CTU.cal_mutual_information(XMS,XMO[:,2]),
                                          CTU.cal_mutual_information(XMS,XBC),
                                          CTU.cal_mutual_information(XMO[:,2],XBC),
                                          CTU.cal_mutual_information(XBC,XMS),
                                          CTU.cal_mutual_information(XMO[:,2],XMS)])
mutual_info_MOshooting = {}
mutual_info_MOshooting = {label : mutual_info_MOshooting_values[i] for i, label in enumerate(mutual_info_MOshooting_label)} 

mutual_info_MOshooting

{'MO_shooting->BC': 0.0012452430134959336,
 'MO_shooting->MS': 0.006117448119202784,
 'BC->MS': 0.000875914415846557,
 'BC->MO_shooting': 0.0012501595803720011,
 'MS->BC': 0.010884142586891077,
 'MS->MO_shooting': 0.0032017309356648926}

In [19]:
# P_values test (MO_shooting)
sur_mutual_info_shooting_values = [CTU.surrogate_mutual_information(XBC,XMO[:,2],nsurr),
                                   CTU.surrogate_mutual_information(XMS,XMO[:,2],nsurr),
                                   CTU.surrogate_mutual_information(XMS,XBC,nsurr),
                                   CTU.surrogate_mutual_information(XMO[:,2],XBC,nsurr),
                                   CTU.surrogate_mutual_information(XBC,XMS,nsurr),
                                   CTU.surrogate_mutual_information(XMO[:,2],XMS,nsurr)]

sur_mutual_info_shooting = {}
sur_mutual_info_shooting = {label : sur_mutual_info_shooting_values[i] for i, label in enumerate(mutual_info_MOshooting_label)}
 
p_values_shooting = {}
ninetyfive_tab_shooting = {}

for i, label in enumerate(mutual_info_MOshooting_label):
    aux_count_sum = np.sum(sur_mutual_info_shooting[label] > mutual_info_MOshooting[label])
    p_values_shooting[label] = (aux_count_sum + 1) / (nsurr + 1)
    ninetyfive_tab_shooting[label] = np.quantile(sur_mutual_info_shooting[label] , 0.95)
p_values_shooting

{'MO_shooting->BC': 0.5469061876247505,
 'MO_shooting->MS': 0.18163672654690619,
 'BC->MS': 0.5888223552894212,
 'BC->MO_shooting': 0.5848303393213573,
 'MS->BC': 0.0718562874251497,
 'MS->MO_shooting': 0.34331337325349304}